In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import time
from sklearn.ensemble import GradientBoostingClassifier
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from PIL import Image  # Importe a classe Image da biblioteca PIL
# import helper
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader

In [ ]:
import pandas as pd

# Leia o arquivo Excel
df = pd.read_excel('C:/Users/Karol/Documents/Python projeto/microscopy_ground_truth.xlsx')

# Exiba as primeiras linhas do DataFrame para verificar se os dados foram lidos corretamente
print(df.head())



In [ ]:
print(df.columns)

In [ ]:
classes = ('Benign', 'InSitu', 'Invasive', 'Normal')

In [ ]:
# Diretório onde estão localizadas as imagens
diretorio_imagens = 'C:/Users/Karol/Documents/Python projeto/ICIAR2018_BACH_Challenge/ICIAR2018_BACH_Challenge/Photos/BACH_Preparado/200x200_pb/'

# Crie uma lista de caminhos para as imagens
image_paths = [diretorio_imagens + nome_da_imagem for nome_da_imagem in df['nome_da_imagem']]


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Instancie o LabelEncoder
label_encoder = LabelEncoder()

# Atribua um número único a cada tipo (rótulo)
df['tipo_numerico'] = label_encoder.fit_transform(df['tipo'])

# Exiba as primeiras linhas do DataFrame com os tipos numéricos
print(df[['tipo', 'tipo_numerico']].head())


In [ ]:
PATH='C:/Users/Karol/Documents/Python projeto/ICIAR2018_BACH_Challenge/ICIAR2018_BACH_Challenge/Photos/BACH_Preparado/200x200_pb'


In [ ]:
transform = transforms.Compose(
                    [transforms.ToTensor(),
                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4


In [ ]:
dataset = ImageFolder(PATH, transform=transform)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=1)



In [ ]:
def imshow(img): 
    img = (img * 0.5) + 0.5  # desnormalizar imagens
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
# Um batch de dados de treino
dataiter = iter(dataloader)
images, labels = next(dataiter)

fig = plt.figure(figsize=(20, 4))
for idx in np.arange(4):
    ax = fig.add_subplot(2, 8 // 2, idx + 1, xticks=[], yticks=[])  # Correção na divisão
    image = images[idx].numpy().transpose((1, 2, 0))  # Reorganize as dimensões da imagem
    image_pil = Image.fromarray((image * 255).astype(np.uint8))  # Converte para objeto PIL
    plt.imshow(image_pil)  # Exibe a imagem PIL
    ax.set_title(classes[labels[idx]])

plt.show()

In [ ]:
images[0].shape

In [ ]:
labels

In [ ]:
inicio = time.time()
fim = time.time()

In [ ]:
print (fim - inicio)

Para calcular o tamanho espacial da saida podemos usar a fórmula ((N − F + 2P) / S) + 1.
onde:
N : Dimensão da imagem de entrada
F : Dimensão do Filtro
P : O padding
S : O stride
Lembrando que um maxpooling de (kernel_size = 2,stride = 2) diminui pela metade a dimensão da imagem nos eixos x e y


In [ ]:
label_encoder = LabelEncoder()

df['tipo_numerico'] = label_encoder.fit_transform(df['tipo'])

# Defina sua arquitetura CNN
class CNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # com base nas dimensões da imagem de entrada (200x200)
        self.fc_input_size = 32 * 50 * 50  # (32 canais * 50 * 50 pixels)
        
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.fc2 = nn.Linear(128, 4)  

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, self.fc_input_size)  # Achatamento para as camadas fully connected
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

cnn = CNN()

In [ ]:
#Caso esteja disponível, processar na GPU
USAR_GPU = True 

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.001, momentum=0.9)

# train_loader=torch.utils.data.DataLoader(dataset, batch_size=4)
# optimizer = optim.Adam(cnn.parameters(), lr=0.01)


In [ ]:
use_cuda = USAR_GPU and torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

print('Usar GPU: ', USAR_GPU)
print('GPU disponivel:', torch.cuda.is_available())
print('Processando em:', device )

#Seta onde será executada
cnn.to(device)

In [ ]:
x = []
y_acertos = []
y_loss = []
epochs = 40
inicio = time.time()

# Inicie o treinamento
for epoch in range(epochs):
    total_loss = 0
    total_acertos = 0
    
    # Itere pelo DataLoader
    for images, labels in dataloader:
        #Coloca os dados no dispositivo de processamento
        images, labels = images.to(device), labels.to(device)

        # Restante do código para treinamento da rede
        preds = cnn(images)
        loss = F.cross_entropy(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Cálculos de acertos e perda
        acertos = preds.argmax(dim=1).eq(labels).sum().item()
        total_loss += loss.item()
        total_acertos += acertos
    
    # Armazene as métricas para plotagem ou acompanhamento
    x.append(epoch)
    y_acertos.append(total_acertos)
    y_loss.append(total_loss)

    # Exiba o progresso
    print('epoch:', epoch, end='')
    print('\tloss: ', round(total_loss, 2), end='')
    print('\tacertos:', total_acertos, f'({round(total_acertos * 100 / len(df), 2)}%)')

print("Treinamento concluído!")
fim = time.time()
tempo = round(fim - inicio, 2)
minutos = tempo // 60
segundos = round (tempo % 60,2)
print(minutos,'minutos e' , segundos, 'segundos' )
print(tempo, 'segundos')


In [ ]:
import os
model_path = './model/'
file_name = 'cnn_model.pth'

if not os.path.exists(model_path): 
        os.makedirs(model_path)
        
torch.save(cnn.state_dict(), model_path + file_name)  

In [ ]:
# Um batch de dados de treino
dataiter = iter(dataloader)
images, labels = next(dataiter)

fig = plt.figure(figsize=(20, 4))
for idx in np.arange(4):
    ax = fig.add_subplot(2, 8 // 2, idx + 1, xticks=[], yticks=[])  # Correção na divisão
    image = images[idx].numpy().transpose((1, 2, 0))  # Reorganize as dimensões da imagem
    image_pil = Image.fromarray((image * 255).astype(np.uint8))  # Converte para objeto PIL
    plt.imshow(image_pil)  # Exibe a imagem PIL
    ax.set_title(classes[labels[idx]])

plt.show()

In [ ]:
cnn = CNN()
cnn.load_state_dict(torch.load(model_path + file_name))

In [ ]:
outputs = cnn(images)
outputs

_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))



In [ ]:
correct = 0
total = 0
# nao precisamos calcular os valores de gradiente pq não estamos treinando
with torch.no_grad():
    for data in dataloader:
        images, labels = data
        outputs = cnn(images)
        # a classe com a maior energia é o que escolhemos como previsão
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

In [ ]:
# contar as previsões para cada classe
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# novamente sem gradientes necessários
with torch.no_grad():
    for data in dataloader:
        images, labels = data
        outputs = cnn(images)
        _, predictions = torch.max(outputs, 1)
        # calcula predições corretas para cada classe
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print acurácia para cada classe
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))

# class TestSquare(TestCase): 
#     def test_classe(self): 
#         result = accuracy
#         expected = 90.0
#         self.assertEqual(result, expected)     
#         print(classname, accuracy)
# if __name__ == '__main__':
#     s = unittest.TestLoader().loadTestsFromTestCase(TestSquare)
#   unittest.TextTestRunner().run(s)

 

- Porcentagem % -  método para testar,
para saber a porcentagem para validar se
está acima ou abaixo do limite estipulado

- E quantidade de acertos em número

- Validar se está passando por
todas as imagens

- Validar a acurácia de cada tipo
 ('Benign', 'InSitu', 'Invasive', 'Normal')
e ver qual tipo teve maior acerto


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(x,y_acertos)
#plt.ylim(bottom=0)
plt.show()

In [ ]:
plt.plot(x,y_loss)
#plt.ylim(bottom=0) 
plt.show()

REALIZANDO TESTES

In [ ]:
from PIL import Image
import torchvision.transforms as transforms

# image_path = 'C:/Users/Karol/Documents/Python projeto/ICIAR2018_BACH_Challenge/ICIAR2018_BACH_Challenge/Photos/BACH_Preparado/200x200_pb/Normal/n003.tif'
image_path = 'C:/Users/Karol/Documents/Python projeto/ICIAR2018_BACH_Challenge/ICIAR2018_BACH_Challenge/Photos/Invasive/iv008.tif'

image = Image.open(image_path)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((200, 200)),  # Redimensionar para o tamanho de entrada da rede
    transforms.Grayscale(num_output_channels=3),  # Converter para escala de cinza com três canais
    transforms.ToTensor(),  # Converter para tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizar
])
input_image = transform(image).unsqueeze(0)  # Adicionar uma dimensão de lote (batch) única

In [ ]:
with torch.no_grad():  # Desativar o cálculo de gradientes para economizar memória
    outputs = cnn(input_image)
    _, predicted = torch.max(outputs, 1)

In [ ]:
classe_predita = classes[predicted.item()]
print(f'A classe prevista para a imagem é: {classe_predita}')